In [1]:
# NFL play visualization
# Load data set
# use time, x, and y to plot the play
# use team, player, and position to color the plot
from dash import Dash, html, dcc
import plotly.express as px
import plotly.tools as tls
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
import os   
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from dash import Dash, html, dcc, Input, Output, State
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import sequential
import zipfile

from dash import callback_context
# import nfl_data_py as nfl


# #load data tracking
# wk1 = pd.read_csv(r"C:\Users\Collin\OneDrive - Gettysburg College\Gettysburg Spring 2025\NFL DATA\nfl-big-data-bowl-2025\tracking_week_1.csv")
# #load player info
# player_info = pd.read_csv(r"C:\Users\Collin\OneDrive - Gettysburg College\Gettysburg Spring 2025\NFL DATA\nfl-big-data-bowl-2025\players.csv")
# #load game data
# game_data = pd.read_csv(r"C:\Users\Collin\OneDrive - Gettysburg College\Gettysburg Spring 2025\NFL DATA\nfl-big-data-bowl-2025\games.csv")
# #load play data
# play_data = pd.read_csv(r"C:\Users\Collin\OneDrive - Gettysburg College\Gettysburg Spring 2025\NFL DATA\nfl-big-data-bowl-2025\plays.csv")

In [2]:
# Check if the dataset is already downloaded
if not os.path.exists('nfl-big-data-bowl-2025.zip'):
    # Download the dataset using Kaggle API
    os.system('kaggle competitions download -c nfl-big-data-bowl-2025')

# Extract the dataset
if not os.path.exists('nfl_data'):
    with zipfile.ZipFile('nfl-big-data-bowl-2025.zip', 'r') as zip_ref:
        zip_ref.extractall('nfl_data')

# Load the dataset
wk1 = pd.read_csv('nfl_data/tracking_week_1.csv')
player_info = pd.read_csv('nfl_data/players.csv')
game_data = pd.read_csv('nfl_data/games.csv')
play_data = pd.read_csv('nfl_data/plays.csv')

print(wk1.head())

       gameId  playId    nflId     displayName  frameId    frameType  \
0  2022091200      64  35459.0  Kareem Jackson        1  BEFORE_SNAP   
1  2022091200      64  35459.0  Kareem Jackson        2  BEFORE_SNAP   
2  2022091200      64  35459.0  Kareem Jackson        3  BEFORE_SNAP   
3  2022091200      64  35459.0  Kareem Jackson        4  BEFORE_SNAP   
4  2022091200      64  35459.0  Kareem Jackson        5  BEFORE_SNAP   

                    time  jerseyNumber club playDirection      x      y     s  \
0  2022-09-13 00:16:03.5          22.0  DEN         right  51.06  28.55  0.72   
1  2022-09-13 00:16:03.6          22.0  DEN         right  51.13  28.57  0.71   
2  2022-09-13 00:16:03.7          22.0  DEN         right  51.20  28.59  0.69   
3  2022-09-13 00:16:03.8          22.0  DEN         right  51.26  28.62  0.67   
4  2022-09-13 00:16:03.9          22.0  DEN         right  51.32  28.65  0.65   

      a   dis       o    dir                 event  
0  0.37  0.07  246.17  68.3

In [3]:
wk1.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022091200,64,35459.0,Kareem Jackson,1,BEFORE_SNAP,2022-09-13 00:16:03.5,22.0,DEN,right,51.06,28.55,0.72,0.37,0.07,246.17,68.34,huddle_break_offense
1,2022091200,64,35459.0,Kareem Jackson,2,BEFORE_SNAP,2022-09-13 00:16:03.6,22.0,DEN,right,51.13,28.57,0.71,0.36,0.07,245.41,71.21,NaN
2,2022091200,64,35459.0,Kareem Jackson,3,BEFORE_SNAP,2022-09-13 00:16:03.7,22.0,DEN,right,51.20,28.59,0.69,0.23,0.07,244.45,69.90,NaN
3,2022091200,64,35459.0,Kareem Jackson,4,BEFORE_SNAP,2022-09-13 00:16:03.8,22.0,DEN,right,51.26,28.62,0.67,0.22,0.07,244.45,67.98,NaN
4,2022091200,64,35459.0,Kareem Jackson,5,BEFORE_SNAP,2022-09-13 00:16:03.9,22.0,DEN,right,51.32,28.65,0.65,0.34,0.07,245.74,62.83,NaN


In [4]:
# Filter for the 'pass_forward' event
pass_plays = wk1[(wk1['event'] == 'pass_forward') & (wk1['frameType'] == 'AFTER_SNAP')]

pass_plays
# List to store passers
passers = []

for _, play in pass_plays.groupby(['gameId', 'playId']):
    # Exclude players with jersey numbers over 15
    # play = play[play['jerseyNumber'] <= 19]

    # Filter out the football data
    football_data = play[play['displayName'] == 'football']
    if football_data.empty:
        continue

    # Get the football's position at the time of the pass
    football_x = football_data.iloc[0]['x']
    football_y = football_data.iloc[0]['y']

    # Calculate the distance of each player to the football
    play['distance_to_football'] = np.sqrt((play['x'] - football_x)**2 + (play['y'] - football_y)**2)

    # Find the player closest to the football
    closest_player = play[play['displayName'] != 'football'].nsmallest(1, 'distance_to_football')
    # print(closest_player)

    if not closest_player.empty:
        passers.append({
            'playId': play['playId'].iloc[0],
            'gameId': play['gameId'].iloc[0],
            'jersey_number': closest_player.iloc[0]['jerseyNumber'],
            'club': closest_player.iloc[0]['club'],
            'passer_name': closest_player.iloc[0]['displayName']
        })

# passers

passers_df = pd.DataFrame(passers)
passers_df.head()



,playId,gameId,jersey_number,club,passer_name
0,56,2022090800,17.0,BUF,Josh Allen
1,122,2022090800,17.0,BUF,Josh Allen
2,167,2022090800,17.0,BUF,Josh Allen
3,212,2022090800,17.0,BUF,Josh Allen
4,236,2022090800,17.0,BUF,Josh Allen


In [5]:
print(wk1['event'].unique())

['huddle_break_offense' nan 'line_set' 'man_in_motion' 'ball_snap'
 'handoff' 'first_contact' 'tackle' 'pass_forward' 'pass_arrived'
 'pass_outcome_caught' 'touchdown' 'dropped_pass'
 'pass_outcome_incomplete' 'play_action' 'out_of_bounds' 'run' 'qb_sack'
 'pass_tipped' 'fumble' 'fumble_offense_recovered' 'shift'
 'fumble_defense_recovered' 'touchback' 'qb_strip_sack' 'qb_kneel'
 'timeout_away' 'snap_direct' 'pass_outcome_interception' 'qb_slide'
 'play_submit' 'pass_outcome_touchdown' 'lateral' 'qb_spike' 'pass_shovel'
 'run_pass_option' 'huddle_start_offense']


In [6]:
passers_df = passers_df[passers_df['jersey_number'] <= 19]
passers_df

,playId,gameId,jersey_number,club,passer_name
0,56,2022090800,17.0,BUF,Josh Allen
1,122,2022090800,17.0,BUF,Josh Allen
2,167,2022090800,17.0,BUF,Josh Allen
3,212,2022090800,17.0,BUF,Josh Allen
4,236,2022090800,17.0,BUF,Josh Allen
...,...,...,...,...,...
1071,3574,2022091200,3.0,DEN,Russell Wilson
1072,3596,2022091200,3.0,DEN,Russell Wilson
1073,3628,2022091200,3.0,DEN,Russell Wilson
1075,3747,2022091200,3.0,DEN,Russell Wilson


In [7]:
# Filter passers_df to keep only passer_names that appear 3 or more times
passers_df = passers_df[passers_df['passer_name'].map(passers_df['passer_name'].value_counts()) >= 4]
passers_df

,playId,gameId,jersey_number,club,passer_name
0,56,2022090800,17.0,BUF,Josh Allen
1,122,2022090800,17.0,BUF,Josh Allen
2,167,2022090800,17.0,BUF,Josh Allen
3,212,2022090800,17.0,BUF,Josh Allen
4,236,2022090800,17.0,BUF,Josh Allen
...,...,...,...,...,...
1071,3574,2022091200,3.0,DEN,Russell Wilson
1072,3596,2022091200,3.0,DEN,Russell Wilson
1073,3628,2022091200,3.0,DEN,Russell Wilson
1075,3747,2022091200,3.0,DEN,Russell Wilson


In [8]:
# #select game based on quarterback, as well as plays
# # Filter the passers_df for the selected quarterback
# selected_quarterback = "Josh Allen"  # Replace with the desired quarterback's name
# qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]
# qb_plays_df #this is pass plays already

# # Filter the wk1 dataframe for rows where the displayName is 'football' and the playId is in qb_plays_df
# football_locations = wk1[(wk1['displayName'] == 'football') & (wk1['playId'].isin(qb_plays_df['playId'])) & (wk1['gameId'].isin(qb_plays_df['gameId']))]
# football_locations

# football_locations = football_locations[football_locations['event'] == 'pass_arrived']
# football_locations

In [18]:

app = Dash(__name__, suppress_callback_exceptions=True)
game_id = 2022091106
selected_quarterback = "Josh Allen"  # Replace with the desired quarterback's name

#team color map:
colormap={  # AI colors for each team
            'ARI': '#97233F', 'ATL': '#A71930', 'BAL': '#241773', 'BUF': '#00338D',
            'CAR': '#0085CA', 'CHI': '#C83803', 'CIN': '#FB4F14', 'CLE': '#ff3c00',
            'DAL': '#003594', 'DEN': '#FB4F14', 'DET': '#0076B6', 'GB': '#203731',
            'HOU': '#03202F', 'IND': '#002C5F', 'JAX': '#006778', 'KC': '#E31837', 'LA': '#ff8200',
            'LAC': '#0080C6', 'LAR': '#003594', 'LV': '#000000', 'MIA': '#008E97',
            'MIN': '#4F2683', 'NE': '#B0B7BC', 'NO': '#D3BC8D', 'NYG': '#0B2265',
            'NYJ': '#125740', 'PHI': '#046A38', 'PIT': '#FFB612', 'SEA': '#69BE28',
            'SF': '#AA0000', 'TB': '#D50A0A', 'TEN': '#4B92DB', 'WAS': '#773141',
            'football': 'yellow'
        }

pass_result_labels = {
    'C': 'Complete',
    'I': 'Incomplete',
    'S': 'Sack',
    'IN': 'Intercepted',
    'R': 'Scramble'
}

# Colormap for pass outcomes
pass_outcome_colormap = {
    'Complete': 'green',       # Complete pass
    'Incomplete': 'red',         # Incomplete pass
    'Sack': 'orange',      # Quarterback sack
    'Intercepted': 'purple',     # Intercepted pass
    'Scramble': 'blue'         # Scramble
}


#App layout
app.layout = html.Div([
    dcc.Dropdown(
        id='quarterback-selector',
        options=[{'label': qb, 'value': qb} for qb in passers_df['passer_name'].unique()],
        value=passers_df['passer_name'].iloc[0],  # Default value
        placeholder="Select a quarterback",
        style={'width': '50%', 'margin-bottom': '10px'}  # Adjust the width and spacing of dropdowns
    ),
    # Not needed for now, but can be used to select games. Keeping incase I expand dataset to include more games
    # dcc.Dropdown(
    #     id='game-selector',
    #     options=[],  # Initially empty; will be updated dynamically
    #     placeholder="Select a game",
    #     style={'width': '50%', 'margin-bottom': '10px'}  # Adjust the width and spacing of dropdowns
    # ),
    dcc.Dropdown(
        id='play-selector',
        options=[],  # Initially empty; will be updated dynamically
        placeholder="Select a play",
        style={'width': '50%', 'margin-bottom': '10px'}  # Adjust the width and spacing of dropdowns
    ),
    dcc.Graph(
        id='play-visualization'
    ),
    html.Button("Clear Traces", id='clear-traces-button'),  # Add a button to clear traces
    dcc.Store(id='selected-players', data=[]),
    dcc.Store(id='selected-play-id', data=None),  # Store to keep track of selected players
    # dcc.Graph(id='kde-plot', style={'width': '80%', 'margin-top': '20px'}),  # Add the KDE plot
    # dcc.Graph(id='scatter-plot', style={'width': '80%', 'margin-top': '20px'})  # Add the scatterplot
    html.Div([
        dcc.Graph(id='scatter-plot', style={'flex': '1', 'margin-right': '10px'}),  # Scatterplot
        dcc.Graph(id='kde-plot', style={'flex': '1'})  # Heatmap
    ], style={'display': 'flex', 'flex-direction': 'row', 'width': '100%'})  # Flexbox for side-by-side layout
])



@app.callback(
    Output('play-selector', 'value'),  # Update the play-selector dropdown value
    [Input('scatter-plot', 'clickData')],  # Listen to scatterplot click events
    prevent_initial_call=True
)
def sync_play_selector_with_scatterplot(click_data):
    if click_data and 'points' in click_data:
        # Extract the playId from the clicked point's hovertext
        selected_play_id = click_data['points'][0]['hovertext']
        return selected_play_id
    return dash.no_update  # Do not update if no clickData is available

@app.callback(
    Output('play-selector', 'options'),
    [Input('quarterback-selector', 'value')],
    prevent_initial_call=True
)
def update_play_selector(selected_quarterback):
    # Ensure selected_quarterback is passed as an argument
    qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]

    # Get the unique game ID (since there's only one game per quarterback)
    if qb_plays_df.empty:
        return []

    game_id = qb_plays_df['gameId'].iloc[0]

    # Get the unique play IDs for the selected quarterback
    play_ids = qb_plays_df['playId'].unique()

    # Create options for the play-selector dropdown
    play_options = [{'label': f"Play {play_id}", 'value': play_id} for play_id in play_ids]

    return play_options



# Update the selected quarterback based on the dropdown
@app.callback(
    Output('quarterback-selector', 'value'),
    [Input('quarterback-selector', 'value')]
)
def update_selected_quarterback(selected_qb):
    global selected_quarterback
    selected_quarterback = selected_qb
    return selected_qb
    return [{'label': f"Play {play_id}", 'value': play_id} for play_id in plays]

qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]
# Filter the wk1 dataframe for rows where the displayName is 'football' and the playId is in qb_plays_df
football_locations = wk1[(wk1['displayName'] == 'football') & (wk1['playId'].isin(qb_plays_df['playId'])) & (wk1['gameId'].isin(qb_plays_df['gameId']))]

football_locations = football_locations[football_locations['event'] == 'pass_arrived']


# # Callback to update the scatterplot
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('selected-play-id', 'data')],  # Store the selected playId
    [Input('quarterback-selector', 'value')],
    [Input('scatter-plot', 'clickData')],  # Capture clickData
    prevent_initial_call=True
)
def update_scatter_plot(selected_quarterback, click_data):
    ctx = callback_context
    triggered_input = ctx.triggered[0]['prop_id'].split('.')[0]




    # Filter the passers_df for the selected quarterback
    qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]

    ball_snap = wk1[
        wk1['playId'].isin(qb_plays_df['playId']) &
        (wk1['club'] == qb_plays_df['club'].iloc[0]) &
        (wk1['event'] == 'ball_snap') &
        (wk1['gameId'].isin(qb_plays_df['gameId']))
    ]
  
    # Ensure ball_snap is grouped by playId
    ball_snap_grouped = ball_snap.groupby('playId')

    play_df = play_data[(play_data['gameId'].isin(qb_plays_df['gameId']))]
  
    play_df.loc[:, 'relativeX'] = play_df.apply(
        lambda row: row['targetX'] - ball_snap_grouped.get_group(row['playId'])['x'].iloc[0]
        if row['playId'] in ball_snap_grouped.groups else None,
        axis=1
    )

    play_df['relativeX'] = play_df['relativeX'].abs()
    play_df = play_df.merge(qb_plays_df[['playId', 'club']], on='playId', how='left')
    play_df['passResult'] = play_df['passResult'].map(pass_result_labels)

    # Create the scatterplot
    fig = px.scatter(
        play_df,
        y='relativeX',  # Adjust x-coordinates relative to the quarterback
        x='targetY',
        color='passResult',
        color_discrete_map=pass_outcome_colormap,  # Set color based on club using colormap
        title=f"Pass Locations for {selected_quarterback}",
        labels={'x': 'X Coordinate', 'y': 'Y Coordinate'},
        hover_name='playId',
        template='plotly_dark'
        )

    fig.update_layout(
        # yaxis=dict(title='Field Length', range=[-10, 120]),
        yaxis=dict(title='Field Length', range=[-10, 10 + play_df['relativeX'].max()], zeroline=False),
        xaxis=dict(title='Field Width', range=[0, 53.3], zeroline=False),
        xaxis_showgrid=False,  # Remove x-axis grid lines
        yaxis_showgrid=False,  # Remove y-axis grid lines
    
        shapes = [
        dict(
            type='line',
            x0=0,
            x1=53,

            y0=x,
            y1 = x,
            layer='below',
            line=dict(color='white', width=1)
        ) for x in range(0, 120, 10)
    ] + [
       #HASMARKS
        dict(
            type='line',
            x0=26.65 - 5,
            x1=26.65 - 5,
            y0=-10,
            y1=120,
            line=dict(color='white', width=1, dash='dash'),
            layer='below'
        )
    ] + [
      
        dict(
            type='line',
            x0=26.65 + 5,
            x1=26.65 + 5,
            y0=-10,
            y1=120,
            line=dict(color='white', width=1, dash='dash'),
            layer='below'
        )
    ] + [dict(
        type='line',
            x0=0,
            x1=53,
            y0=0,
            y1 = 0,
            layer='below',
            line=dict(color='blue', width=1)
    )],
        plot_bgcolor='#135a27',
        legend_title="Pass Outcome",
        yaxis_scaleanchor="y",
        # xaxis_scaleanchor="y"
        width = 500

    )

    fig.update_traces(
        marker=dict(
            size=10,  # Marker size
            line=dict(
                width=1,  # Outline width
                color='black'  # Outline color
            )
        )
    )


    selected_play_id = None
    if click_data and 'points' in click_data:
        selected_play_id = click_data['points'][0]['hovertext']  # Extract playId from hovertext


    return fig, selected_play_id 


#KDE PLOT
### KDE PLOTS###
@app.callback(
    Output('kde-plot', 'figure'),
    [Input('quarterback-selector', 'value')],
    prevent_initial_call=True
)
def update_kde_plot(selected_quarterback):
   # Filter the passers_df for the selected quarterback
    qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]
    # print(qb_plays_df)
    # Filter the wk1 dataframe for rows where the displayName is 'football' and the playId is in qb_plays_df

    ball_snap = wk1[
        wk1['playId'].isin(qb_plays_df['playId']) &
        (wk1['club'] == qb_plays_df['club'].iloc[0]) &
        (wk1['event'] == 'ball_snap') &
        (wk1['gameId'].isin(qb_plays_df['gameId']))
    ]
  
    # Ensure ball_snap is grouped by playId
    ball_snap_grouped = ball_snap.groupby('playId')

    play_df = play_data[(play_data['gameId'].isin(qb_plays_df['gameId']))]
  
    play_df.loc[:, 'relativeX'] = play_df.apply(
        lambda row: row['targetX'] - ball_snap_grouped.get_group(row['playId'])['x'].iloc[0]
        if row['playId'] in ball_snap_grouped.groups else None,
        axis=1
    )

    play_df['relativeX'] = play_df['relativeX'].abs()
    play_df = play_df.merge(qb_plays_df[['playId', 'club']], on='playId', how='left')
    play_df['passResult'] = play_df['passResult'].map(pass_result_labels)


   # Create the KDE plot using Plotly
    fig = go.Figure(go.Histogram2dContour(
    y=play_df['relativeX'],  # Pass the x values directly
    x=play_df['targetY'],    # Pass the y values directly
    colorscale= 
    [
    [0.0, "rgba(0, 255, 255, 0)"],  # Transparent cyan at 0 (lowest)
    [0.25, "rgba(0, 255, 255, 0.4)"],  # Cyan
    [0.5, "rgba(0, 255, 128, 0.6)"],  # Greenish-cyan
    [0.75, "rgba(255, 255, 0, 0.8)"],  # Yellow
    [1.0, "rgba(255, 0, 0, 1)"]  # Red (highest)
]
,
    contours=dict(coloring="fill", showlabels=True)
))

# Update layout
    fig.update_layout(
        title=f'Heatmap of targets for {selected_quarterback}',
        yaxis=dict(title='Field Length', range=[-10, 10 + play_df['relativeX'].max()], zeroline=False),
        xaxis=dict(title='Field Width', range=[0, 53], zeroline=False),
        plot_bgcolor='#135a27',
        xaxis_showgrid=False,  # Remove x-axis grid lines
        yaxis_showgrid=False,  # Remove y-axis grid lines
        shapes = [
        dict(
            type='line',
            x0=0,
            x1=53,
            y0=x,
            y1 = x,
            layer='below',
            line=dict(color='white', width=1)
        ) for x in range(10, 120, 10)
    ]+ [dict(
        type='line',
            x0=0,
            x1=53,
            y0=0,
            y1 = 0,
            layer='below',
            line=dict(color='blue', width=1)
    )] + [
       #HASMARKS
        dict(
            type='line',
            x0=26.65 - 5,
            x1=26.65 - 5,
            y0=-10,
            y1=120,
            line=dict(color='white', width=1, dash='dash'),
            layer='below'
        )
    ] + [
      
        dict(
            type='line',
            x0=26.65 + 5,
            x1=26.65 + 5,
            y0=-10,
            y1=120,
            line=dict(color='white', width=1, dash='dash'),
            layer='below'
        )
    ] 
    ,
        template='plotly_dark',
        yaxis_scaleanchor="y",
        width = 500
    )
    

    return fig


    # ctx = callback_context
    # triggered_input = ctx.triggered[0]['prop_id'].split('.')[0]

    # # Case 1: Clear all traces when the "Clear Traces" button is clicked
    # if triggered_input == 'clear-traces-button':
    #     if not current_figure or 'data' not in current_figure:
    #         return dash.no_update  # Do nothing if the figure is empty
    #     current_figure['data'] = []  # Clear all traces
    #     return current_figure

    # # Case 2: Add a trace for the clicked player
    # if triggered_input == 'play-visualization':
    #     if not click_data or not selected_play_id:
    #         return dash.no_update

    #     # Extract the clicked player's name from the scatterplot
    #     clicked_player = click_data['points'][0]['hovertext']  # Assuming hovertext contains the player's name

    #     # Get the game ID for the selected quarterback
    #     qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]
    #     if qb_plays_df.empty:
    #         return dash.no_update
    #     game_id = qb_plays_df['gameId'].iloc[0]

    #     # Filter the data for the selected play and player
    #     play = wk1[(wk1['playId'] == selected_play_id) & (wk1['gameId'] == game_id)]
    #     player_data = play[play['displayName'] == clicked_player]

    #     if player_data.empty:
    #         return dash.no_update

    #     # Create a trace for the player's movement
    #     player_trace = dict(
    #         type='scatter',
    #         x=player_data['x'],
    #         y=player_data['y'],
    #         mode='lines+markers',
    #         line=dict(color='red', width=2),  # Customize the trace color and width
    #         marker=dict(size=6),
    #         name=f"{clicked_player} Movement",
    #         hoverinfo='text',
    #         text=[
    #             f"Frame: {frame}, X: {x}, Y: {y}"
    #             for frame, x, y in zip(player_data['frameId'], player_data['x'], player_data['y'])
    #         ]
    #     )

    #     # Add the new trace to the existing figure
    #     if 'data' not in current_figure:
    #         current_figure['data'] = []
    #     current_figure['data'].append(player_trace)

    #     return current_figure

    # return dash.no_update




@app.callback(
    Output('play-visualization', 'figure'),
    [Input('play-selector', 'value'),  # Selected play ID
     Input('quarterback-selector', 'value'),  # Selected quarterback
     Input('play-visualization', 'clickData'),  # Clicked player data
     Input('clear-traces-button', 'n_clicks')],  # Clear traces button
    State('play-visualization', 'figure'),  # Current figure state
    prevent_initial_call=True
)
def update_play_visualization_or_trace(selected_play_id, selected_quarterback, click_data, clear_clicks, current_figure):
    ctx = callback_context
    triggered_input = ctx.triggered[0]['prop_id'].split('.')[0]

    # Case 1: Clear all traces when the "Clear Traces" button is clicked
    if triggered_input == 'clear-traces-button':
        if not current_figure or 'data' not in current_figure:
            return dash.no_update  # Do nothing if the figure is empty

        # Keep only the base traces (e.g., traces without "Movement" in their name)
        base_traces = [trace for trace in current_figure['data'] if "Movement" not in trace.get('name', '')]
        current_figure['data'] = base_traces  # Retain only base traces
        return current_figure

    # Case 2: Update the play visualization
    if triggered_input in ['play-selector', 'quarterback-selector']:
        if not selected_play_id:
            return px.scatter(
                 pd.DataFrame({'x': [], 'y': []}),  # Provide an empty DataFrame
                x='x',
                y='y',
                title="No play selected",
                template='plotly_dark'
            )

        # Get the game ID for the selected quarterback
        qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]
        if qb_plays_df.empty:
            return px.scatter(
                title="No data available for the selected quarterback",
                template='plotly_dark'
            )
        game_id = qb_plays_df['gameId'].iloc[0]

        # Filter the data for the selected play
        play = wk1[(wk1['playId'] == selected_play_id) & (wk1['gameId'] == game_id)]
        if play.empty:
            return px.scatter(
                title="No data available for the selected play",
                template='plotly_dark'
            )

        # Identify the frame where the ball is snapped
        ball_snap_data = play[play['event'] == 'ball_snap']
        if ball_snap_data.empty:
            return px.scatter(
                title="No ball_snap event found for the selected play",
                template='plotly_dark'
            )

        ball_snap_x = ball_snap_data['x'].iloc[0]
        game_df = game_data[game_data['gameId'] == game_id]

        # Initialize the scatter plot
        fig = px.scatter(
            play,
            x='x',
            y='y',
            animation_frame='frameId',
            animation_group='displayName',
            color_discrete_map=colormap,
            color='club',
            hover_name='displayName',
            title=f"Player Movement During Play {selected_play_id} on {game_df['gameDate'].iloc[0]}",
            labels={'x': 'X Coordinate', 'y': 'Y Coordinate'},
            template='plotly_dark'
        )

        # Add point outlines
        fig.update_traces(
            marker=dict(
                size=10,  # Marker size
                line=dict(
                    width=1,  # Outline width
                    color='black'  # Outline color
                )
            )
        )

        # Add the line of scrimmage and first down line
        shapes = [
            dict(
                type='line',
                x0=x,
                x1=x,
                y0=0,
                layer='below',
                y1=53.3,
                line=dict(color='white', width=1)
            ) for x in range(10, 120, 10)
        ]
        shapes += [
            # Line of scrimmage
            dict(
                type='line',
                x0=ball_snap_x,
                x1=ball_snap_x,
                y0=0,
                y1=53.3,
                line=dict(color='blue', width=2, dash='solid'),
                layer='below'
            ),
            # Ten-yard line
            dict(
                type='line',
                x0=ball_snap_x + 10 if play['playDirection'].iloc[0] == 'right' else ball_snap_x - 10,
                x1=ball_snap_x + 10 if play['playDirection'].iloc[0] == 'right' else ball_snap_x - 10,
                y0=0,
                y1=53.3,
                line=dict(color='yellow', width=2, dash='solid'),
                layer='below'
            )
        ] 
        shapes += [
    # Left endzone
            dict(
                type='rect',
                x0=0,  # Start of the left endzone
                x1=10,  # End of the left endzone
                y0=0,  # Bottom of the field
                y1=53.3,  # Top of the field
                fillcolor='blue',  # Color of the endzone
                opacity=0.5,  # Transparency
                layer='below',  # Place it below the data
                line=dict(width=0)  # No border
            ),
            # Right endzone
            dict(
                type='rect',
                x0=110,  # Start of the right endzone
                x1=120,  # End of the right endzone
                y0=0,  # Bottom of the field
                y1=53.3,  # Top of the field
                fillcolor='red',  # Color of the endzone
                opacity=0.5,  # Transparency
                layer='below',  # Place it below the data
                line=dict(width=0)  # No border
            )
        ]

        fig.update_layout(
            xaxis=dict(title='X Coordinate', range=[0, 120]),  # Field lines
            yaxis=dict(title='Y Coordinate', range=[0, 53.3]),
            xaxis_showgrid=False,  # Remove x-axis grid lines
            yaxis_showgrid=False,  # Remove y-axis grid lines
            shapes=shapes,
            plot_bgcolor='#135a27',
            legend_title="Players by Club",
            updatemenus=[
                dict(
                    type="buttons",
                    showactive=False,
                    buttons=[
                        dict(
                            label="Play",
                            method="animate",
                            args=[None, {
                                "frame": {"duration": 5, "redraw": True},
                                "fromcurrent": True
                            }]
                        ),
                        dict(
                            label="Pause",
                            method="animate",
                            args=[[None], {
                                "mode": "immediate",
                                "frame": {"duration": 0, "redraw": False}
                            }]
                        )
                    ]
                )
            ],
            xaxis_scaleanchor="x"
        )
        return fig

    # Case 3: Add a trace for the clicked player
    elif triggered_input == 'play-visualization':
        if not click_data or not selected_play_id:
            return dash.no_update

        # Extract the clicked player's name from the scatterplot
        clicked_player = click_data['points'][0]['hovertext']  # Assuming hovertext contains the player's name

        # Get the game ID for the selected quarterback
        qb_plays_df = passers_df[passers_df['passer_name'] == selected_quarterback]
        game_id = qb_plays_df['gameId'].iloc[0]

        # Filter the data for the selected play and player
        play = wk1[(wk1['playId'] == selected_play_id) & (wk1['gameId'] == game_id)]
        player_data = play[play['displayName'] == clicked_player]

        if player_data.empty:
            return dash.no_update

        # Create a trace for the player's movement
        player_trace = dict(
            type='scatter',
            x=player_data['x'],
            y=player_data['y'],
            mode='lines',
            # line=dict(color='red', width=2),  # Customize the trace color and width
            line=dict(color=click_data['points'][0], width=2),  # Set line color to the point's color
            # marker=dict(size=6),
            name=f"{clicked_player} Movement",
            hoverinfo='text',
            text=[
                f"Frame: {frame}, X: {x}, Y: {y}"
                for frame, x, y in zip(player_data['frameId'], player_data['x'], player_data['y'])
            ]
        )

        # Add the new trace to the existing figure
        if 'data' not in current_figure:
            current_figure['data'] = []
        current_figure['data'].append(player_trace)

        return current_figure

    return dash.no_update

#run the app
if __name__ == '__main__':
    app.run(debug=True)

# print(qb_plays_df)
# print(passers_df)


C:\Users\Collin\AppData\Local\Temp\ipykernel_23072\2256370790.py:283: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Collin\AppData\Local\Temp\ipykernel_23072\2256370790.py:289: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Collin\AppData\Local\Temp\ipykernel_23072\2256370790.py:159: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.